# Webscraping Data

Geizhals PC Komponenten

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [61]:

base_url = 'https://geizhals.at'
component_paths = {
    "AMD_CPU": "/?cat=cpuamdam4",
    "Intel_CPU": "/?cat=cpu1151",
    "GPU": "/?cat=gra16_512",
    "SSD": "/?cat=hdssd",
    "RAM": "/?cat=ramddr3"
}




In [62]:
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.google.com/"
}


for component, path in component_paths.items():
    url = base_url + path
    print(f"Scraping {component} from {url}...")

    response = requests.get(url, headers=headers)
    print(f"→ Status Code: {response.status_code}")
    
    time.sleep(1)


Scraping AMD_CPU from https://geizhals.at/?cat=cpuamdam4...
→ Status Code: 200
Scraping Intel_CPU from https://geizhals.at/?cat=cpu1151...
→ Status Code: 200
Scraping GPU from https://geizhals.at/?cat=gra16_512...
→ Status Code: 200
Scraping SSD from https://geizhals.at/?cat=hdssd...
→ Status Code: 200
Scraping RAM from https://geizhals.at/?cat=ramddr3...
→ Status Code: 200


## Scraping AMD CPU


In [64]:
response = requests.get(base_url+ "/?cat=cpuamdam4", headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
print(response.text[:1000])

<!DOCTYPE HTML><html lang="de"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"><title>Prozessoren (CPUs) AMD Preisvergleich Geizhals Österreich</title><link rel="preconnect" href="https://gzhls.at"><link rel="preconnect" href="https://cdn.cookielaw.org"><script type="importmap">{"integrity":{"https://gzhls.at/gsa/danube.ghp-Bvnqwgnw.js":"sha384-RUNKpoZuKK3IFuKyHIFcASFobXhA4nL486r8HS8BQYXzpbBdI7V+Z2vn7UUmqbSb","https://gzhls.at/gsa/ratings.ghp-BnteCqDa.js":"sha384-O0+XezjAIQlpvlFXwrb+qbME4YycLo6+YHwe9UshgawBTTMC9WXB8OoVTzu8egKP","https://gzhls.at/gsa/filterview.ghp-B3efxBS6.js":"sha384-xBiZRirN/YiVZPN+cplGKEDYbplAF9nzm9VMec7FG7idto75tG95KsODXtyQTcJc","https://gzhls.at/gsa/promotiontooltip.ghp-DJyFGcKk.js":"sha384-am14vr+o1SRgc5fERZlcAbqGwDjahHVXtIyuZwlXKs+akVtfiRoyx+1G5LGemdGR","https://gzhls.at/gsa/collapsiblefilters.ghp-uzIt1KD0.js":"sha384-MXN7Usxaeq9WYZIEMmyZdjp6iXlRUdCJz

In [48]:
import re
cpu_list = []

page = 1
max_pages = 30

while page <= max_pages:
    print(f"Scraping Seite {page}...")
    
    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all('div', class_='productlist__product')

    if not products:
        print(f"Keine Produkte auf Seite {page} gefunden – Schleife wird beendet.")
        break

    for product in products:
        name_container = product.find('div', class_='productlist__name')
        price_container = product.find('div', class_='productlist__price')

        if name_container and price_container:
            title_span = name_container.find('span', class_='notrans')
            price_span = price_container.find('span', class_='notrans')

            title = title_span.get_text(strip=True) if title_span else None
            price_text = price_span.get_text(strip=True) if price_span else None

            if not title or not price_text:
                continue

            price = price_text.replace('€', '').replace('.', '').replace(',', '.')
            try:
                price_float = float(price)
            except ValueError:
                continue

            # Beschreibung holen
            desc_container = product.find('div', class_='productlist__item productlist__description')
            desc_span = desc_container.find('span', class_='notrans') if desc_container else None
            description = desc_span.get_text(strip=True) if desc_span else ""

            # Infos parsen
            match_cores = re.search(r'Kerne:\s*(\d+)', description)
            match_turbo = re.search(r'Turbotakt:\s*([\d.]+)GHz', description)
            match_base = re.search(r'Basistakt:\s*([\d.]+)GHz', description)

            cores = int(match_cores.group(1)) if match_cores else None
            turbo = float(match_turbo.group(1)) if match_turbo else None
            base = float(match_base.group(1)) if match_base else None

            cpu_list.append({
                'Title': title,
                'Price_EUR': price_float,
                'Kerne': cores,
                'Turbotakt_GHz': turbo,
                'Basistakt_GHz': base
            })

    page += 1
    time.sleep(1)

df = pd.DataFrame(cpu_list)
print(f"\nInsgesamt {len(df)} CPUs gefunden")
print(df.head())


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...

Insgesamt 900 CPUs gefunden
                                               Title  Price_EUR  Kerne  \
0  AMD Ryzen 7 9800X3D, 8C/16T, 4.70-5.20GHz, box...     492.00      8   
1  AMD Ryzen 7 5700X3D, 8C/16T, 3.00-4.10GHz, box...     224.88      8   
2  AMD Ryzen 7 7800X3D, 8C/16T, 4.20-5.00GHz, box...     383.09      8   
3  AMD Ryzen 9 9950X3D, 16C/32T, 4.30-5.70GHz, bo...  

In [45]:
from datetime import datetime

#Datum wird automatisch in den Filenamen gespeichert
datum = datetime.today().strftime('%Y-%m-%d')
output_filename = f"geizhals_amd_cpus_{datum}.csv"

# Speichern
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"Datei gespeichert unter: {output_filename}")


Datei gespeichert unter: geizhals_amd_cpus_2025-06-15.csv


## Scraping Intel CPU

In [50]:
response = requests.get(base_url+ "/?cat=cpu1151", headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
print(response.text[:1000])

<!DOCTYPE HTML><html lang="de"><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"><title>Prozessoren (CPUs) Intel Preisvergleich Geizhals Österreich</title><link rel="preconnect" href="https://gzhls.at"><link rel="preconnect" href="https://cdn.cookielaw.org"><script type="importmap">{"imports":{"ghpak/categorylist-legacy/bigSearchImages.ghp.js":"https://gzhls.at/gsa/bigSearchImages.ghp-DIv9YWLL.js","ghpak/chart/chart.ghp.js":"https://gzhls.at/gsa/chart.ghp-BzytsrAS.js","ghpak/dealspage/dealspage.ghp.js":"https://gzhls.at/gsa/dealspage.ghp-DPQjN5FN.js","ghpak/variant/variant.ghp_mobile.js":"https://gzhls.at/gsa/variant.ghp_mobile-DhbqnjVV.js","ghpak/pageheader/navigationtracking.ghp.js":"https://gzhls.at/gsa/navigationtracking.ghp-Cj3EFnKm.js","ghpak/filterwidget/filterwidget.ghp.js":"https://gzhls.at/gsa/filterwidget.ghp-CN2zSNwl.js","ghpak/galleryview/galleryview.ghp.js":"http

In [51]:
import re
cpu_list = []

page = 1
max_pages = 30

while page <= max_pages:
    print(f"Scraping Seite {page}...")
    
    
    
    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all('div', class_='productlist__product')

    if not products:
        print(f"Keine Produkte auf Seite {page} gefunden – Schleife wird beendet.")
        break

    for product in products:
        name_container = product.find('div', class_='productlist__name')
        price_container = product.find('div', class_='productlist__price')

        if name_container and price_container:
            title_span = name_container.find('span', class_='notrans')
            price_span = price_container.find('span', class_='notrans')

            title = title_span.get_text(strip=True) if title_span else None
            price_text = price_span.get_text(strip=True) if price_span else None

            if not title or not price_text:
                continue

            price = price_text.replace('€', '').replace('.', '').replace(',', '.')
            try:
                price_float = float(price)
            except ValueError:
                continue

            # Beschreibung holen
            desc_container = product.find('div', class_='productlist__item productlist__description')
            desc_span = desc_container.find('span', class_='notrans') if desc_container else None
            description = desc_span.get_text(strip=True) if desc_span else ""

            # Infos parsen
            match_cores = re.search(r'Kerne:\s*(\d+)', description)
            match_turbo = re.search(r'Turbotakt:\s*([\d.]+)GHz', description)
            match_base = re.search(r'Basistakt:\s*([\d.]+)GHz', description)

            cores = int(match_cores.group(1)) if match_cores else None
            turbo = float(match_turbo.group(1)) if match_turbo else None
            base = float(match_base.group(1)) if match_base else None

            cpu_list.append({
                'Title': title,
                'Price_EUR': price_float,
                'Kerne': cores,
                'Turbotakt_GHz': turbo,
                'Basistakt_GHz': base
            })

    page += 1
    time.sleep(1)

df = pd.DataFrame(cpu_list)
print(f"\nInsgesamt {len(df)} CPUs gefunden")
print(df.head())


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...

Insgesamt 900 CPUs gefunden
                                               Title  Price_EUR  Kerne  \
0  Intel Core Ultra 7 265K, 8C+12c/20T, 3.90-5.50...     302.42     20   
1  Intel Core i5-14600K, 6C+8c/20T, 3.50-5.30GHz,...     210.76     14   
2  Intel Core i5-14400, 6C+4c/16T, 2.50-4.70GHz, ...     151.68     10   
3  Intel Core Ultra 9 285K, 8C+16c/24T, 3.70-5.70...  

In [53]:
#Datum wird automatisch in den Filenamen gespeichert
datum = datetime.today().strftime('%Y-%m-%d')
output_filename = f"geizhals_intel_cpus_{datum}.csv"

# Speichern
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"Datei gespeichert unter: {output_filename}")


Datei gespeichert unter: geizhals_intel_cpus_2025-06-15.csv


## Scraping CPU


In [57]:
url_base = 'https://geizhals.at/?cat=ramddr3&p='
ram_list = []

page = 1
max_pages = 30

while page <= max_pages:
    print(f"Scraping Seite {page}...")
    url = url_base + str(page)
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all('div', class_='productlist__product')

    if not products:
        print(f"Keine Produkte auf Seite {page} gefunden – Schleife wird beendet.")
        break

    for product in products:
        name_container = product.find('div', class_='productlist__name')
        price_container = product.find('div', class_='productlist__price')

        if name_container and price_container:
            title_span = name_container.find('span', class_='notrans')
            price_span = price_container.find('span', class_='notrans')

            title = title_span.get_text(strip=True) if title_span else None
            price_text = price_span.get_text(strip=True) if price_span else None

            if not title or not price_text:
                continue

            price = price_text.replace('€', '').replace('.', '').replace(',', '.')
            try:
                price_float = float(price)
            except ValueError:
                continue

            # RAM Typ
            ram_typ = None
            if "DDR5" in title.upper():
                ram_typ = "DDR5"
            elif "DDR4" in title.upper():
                ram_typ = "DDR4"
            elif "DDR3" in title.upper():
                ram_typ = "DDR3"

            # Kapazität
            match_size = re.search(r'(\d+GB)', title)
            kapazitaet = match_size.group(1) if match_size else None

            # Gekürzter Produktname bis zur Kapazität
            short_name = title
            if kapazitaet:
                short_name = title.split(kapazitaet)[0].strip()

            # CAS Latency
            match_cl = re.search(r'(CL[\d\-]+)', title)
            cas_latency = match_cl.group(1) if match_cl else None

            ram_list.append({
                'Produktname': short_name,
                'Kapazitaet': kapazitaet,
                'Preis_EUR': price_float,
                'RAM_Typ': ram_typ,
                'CAS_Latency': cas_latency
            })

    page += 1
    time.sleep(1)

df = pd.DataFrame(ram_list)
print(f"\nInsgesamt {len(df)} RAM-Produkte gefunden.")
print(df.head())


Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...

Insgesamt 900 RAM-Produkte gefunden.
                                   Produktname Kapazitaet  Preis_EUR RAM_Typ  \
0         Kingston FURY Beast schwarz DIMM Kit       32GB     106.79    DDR5   
1  G.Skill Trident Z5 NEO RGB schwarz DIMM Kit       32GB     122.90    DDR5   
2             Kingston FURY Impact SO-DIMM Kit       64GB     155.78    DDR5   
3         Kingston FU

In [58]:
#Datum wird automatisch in den Filenamen gespeichert
datum = datetime.today().strftime('%Y-%m-%d')
output_filename = f"geizhals_ram_{datum}.csv"

# Speichern
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"Datei gespeichert unter: {output_filename}")


Datei gespeichert unter: geizhals_ram_2025-06-15.csv


## Scraping GPU 

In [65]:
import re
gpu_data = []
max_pages = 30

for page in range(1, max_pages + 1):
    url = f"https://geizhals.at/?cat=gra16_512&pg={page}"
    print(f"Scraping Seite {page} ...")
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    products = soup.find_all("article", class_="galleryview__item card")

    if not products:
        print("Keine Produkte auf dieser Seite gefunden.")
        break

    for product in products:
        # Produktname
        name_tag = product.find("h3", class_="galleryview__name")
        name_link = name_tag.find("a") if name_tag else None
        title = name_link.text.strip() if name_link else None

        # Preis
        price_tag = product.find("span", class_="price")
        price_text = price_tag.text.strip() if price_tag else None

        if not title or not price_text:
            continue

        # Preis bereinigen
        try:
            price = float(price_text.replace("€", "").replace(".", "").replace(",", "."))
        except ValueError:
            continue

        # VRAM (z. B. 16GB, 12GB)
        match_vram = re.search(r"(\d{1,2}GB)", title)
        vram = match_vram.group(1) if match_vram else None

        

        # Gekürzter Produktname
        short_name = title.split(",")[0].strip()


        gpu_data.append({
            "Produktname": short_name,
            
            "VRAM": vram,
            "Preis_EUR": price
        })

    time.sleep(1)

# In DataFrame laden
df = pd.DataFrame(gpu_data)
print(f"\nInsgesamt {len(df)} GPUs extrahiert.")
print(df.head())

Scraping Seite 1 ...
Scraping Seite 2 ...
Scraping Seite 3 ...
Scraping Seite 4 ...
Scraping Seite 5 ...
Scraping Seite 6 ...
Scraping Seite 7 ...
Scraping Seite 8 ...
Scraping Seite 9 ...
Scraping Seite 10 ...
Scraping Seite 11 ...
Scraping Seite 12 ...
Scraping Seite 13 ...
Scraping Seite 14 ...
Scraping Seite 15 ...
Scraping Seite 16 ...
Scraping Seite 17 ...
Scraping Seite 18 ...
Scraping Seite 19 ...
Scraping Seite 20 ...
Scraping Seite 21 ...
Scraping Seite 22 ...
Scraping Seite 23 ...
Scraping Seite 24 ...
Scraping Seite 25 ...
Scraping Seite 26 ...
Scraping Seite 27 ...
Scraping Seite 28 ...
Scraping Seite 29 ...
Scraping Seite 30 ...

Insgesamt 898 GPUs extrahiert.
                              Produktname  VRAM  Preis_EUR
0       ASUS Prime GeForce RTX 5070 Ti OC  16GB     879.00
1         ASUS Prime Radeon RX 9070 XT OC  16GB     714.96
2  PowerColor Hellhound Radeon RX 9070 XT  16GB     714.96
3        ASUS Dual GeForce RTX 5060 Ti OC  16GB     442.69
4        Sapphire Puls

In [66]:
#Datum wird automatisch in den Filenamen gespeichert
datum = datetime.today().strftime('%Y-%m-%d')
output_filename = f"geizhals_gpu_{datum}.csv"

# Speichern
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"Datei gespeichert unter: {output_filename}")


Datei gespeichert unter: geizhals_gpu_2025-06-20.csv


## Scraping SSD Hard Drive Space

In [79]:
ssd_data = []
max_pages = 30

for page in range(1, max_pages + 1):
    url = f"https://geizhals.at/?cat=hdssd&pg={page}"
    print(f"Scraping SSDs Seite {page}...")
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # RICHTIG für SSD-Seite: Listenansicht
    products = soup.find_all("div", class_="productlist__product")

    if not products:
        print("Keine Produkte gefunden – abbrechen.")
        break

    for product in products:
        name_container = product.find("div", class_="productlist__name")
        price_container = product.find("div", class_="productlist__price")

        title_span = name_container.find("span", class_="notrans") if name_container else None
        price_span = price_container.find("span", class_="notrans") if price_container else None

        title = title_span.text.strip() if title_span else None
        price_text = price_span.text.strip() if price_span else None

        if not title or not price_text:
            continue

        try:
            price = float(price_text.replace("€", "").replace(".", "").replace(",", "."))
        except ValueError:
            continue

        # Speichergröße
        match_size = re.search(r"(\d+\s?(?:GB|TB))", title.upper())
        storage = match_size.group(1).replace(" ", "") if match_size else None

        # Produktname kürzen beim ersten Komma
        short_name = title.split(",")[0].strip()

        ssd_data.append({
            "Produktname": short_name,
            "Speicher": storage,
            "Preis_EUR": price
        })

    time.sleep(1)

# In DataFrame
df = pd.DataFrame(ssd_data)
print(f"\n {len(df)} SSDs extrahiert.")
print(df.head())

Scraping SSDs Seite 1...
Scraping SSDs Seite 2...
Scraping SSDs Seite 3...
Scraping SSDs Seite 4...
Scraping SSDs Seite 5...
Scraping SSDs Seite 6...
Scraping SSDs Seite 7...
Scraping SSDs Seite 8...
Scraping SSDs Seite 9...
Scraping SSDs Seite 10...
Scraping SSDs Seite 11...
Scraping SSDs Seite 12...
Scraping SSDs Seite 13...
Scraping SSDs Seite 14...
Scraping SSDs Seite 15...
Scraping SSDs Seite 16...
Scraping SSDs Seite 17...
Scraping SSDs Seite 18...
Scraping SSDs Seite 19...
Scraping SSDs Seite 20...
Scraping SSDs Seite 21...
Scraping SSDs Seite 22...
Scraping SSDs Seite 23...
Scraping SSDs Seite 24...
Scraping SSDs Seite 25...
Scraping SSDs Seite 26...
Scraping SSDs Seite 27...
Scraping SSDs Seite 28...
Scraping SSDs Seite 29...
Scraping SSDs Seite 30...

 900 SSDs extrahiert.
                                    Produktname Speicher  Preis_EUR
0                       Samsung SSD 990 PRO 2TB      2TB     158.95
1                       Samsung SSD 990 PRO 4TB      4TB     286.82
2 

In [80]:
#Datum wird automatisch in den Filenamen gespeichert
datum = datetime.today().strftime('%Y-%m-%d')
output_filename = f"geizhals_ssd_{datum}.csv"

# Speichern
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"Datei gespeichert unter: {output_filename}")


Datei gespeichert unter: geizhals_ssd_2025-06-15.csv


## Scraping Steam Hardware Survey

### Scraping videocard



In [7]:
url_steam_gpu= "https://store.steampowered.com/hwsurvey/videocard/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.google.com/"
}

response = requests.get(url_steam_gpu, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")




# STEAM WEB API + Steamcharts


In [115]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Steam Web API Key
api_key = "D3E772B90A3191FE80299DD4103E287C"
TOP_N = 20

url = "https://steamcharts.com/top"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
print(f"Status Code: {response.status_code}")
soup = BeautifulSoup(response.text, "html.parser")

# Alle Zeilen mit Klasse "odd" oder "even" extrahieren
rows = soup.select("table#top-games tr.odd, table#top-games tr.even")

games = []

for row in rows[:TOP_N]:
    app_tag = row.select_one("td.game-name a")
    player_tag = row.select_one("td.num")

    if app_tag and player_tag:
        name = app_tag.text.strip()
        app_href = app_tag["href"]
        app_id = app_href.split("/")[2]
        player_count = int(player_tag.text.strip().replace(",", ""))

        games.append({
            "app_id": int(app_id),
            "name": name,
            "steamcharts_players": player_count
        })

# Spielerzahlen über Steam Web API
for game in games:
    time.sleep(1)
    url = f"https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?key={api_key}&appid={game['app_id']}"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        game['api_player_count'] = r.json().get('response', {}).get('player_count', 0)
    except Exception as e:
        print(f"Fehler bei AppID {game['app_id']}: {e}")
        game['api_player_count'] = 0

# DataFrame erstellen
df = pd.DataFrame(games)
df = df.sort_values(by="api_player_count", ascending=False).reset_index(drop=True)
print(df)


Status Code: 200
     app_id                       name  steamcharts_players  api_player_count
0       730           Counter-Strike 2              1509405           1296289
1       570                     Dota 2               641549            607429
2   3419430                  Bongo Cat               173192            170455
3    238960              Path of Exile               145294            147892
4    252490                       Rust               131136            133076
5   2622380      ELDEN RING NIGHTREIGN               147849            133007
6    271590  Grand Theft Auto V Legacy               112862            110227
7   2923300                     Banana                84614             84956
8   2767030              Marvel Rivals                75776             80185
9       480                   Spacewar                81796             77333
10  2669320            EA SPORTS FC 25                72218             70796
11  1938090              Call of Duty®         